In [ ]:
!pip install -q gradio requests
!pip install requests
!pip install -q sympy numpy matplotlib


In [ ]:
import requests
import gradio as gr
import json

In [ ]:
SERPER_API_KEY = "285764879e043239ecf494f8bbded332bbceb233"
NEWSDATA_API_KEY = "pub_d28fe66b9a0f459da7e8f8a65a7920e7"

In [ ]:
def google_search(query):
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    data = {"q": query}
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        results = response.json()
        if "organic" in results and len(results["organic"]) > 0:
            return results["organic"][0]["snippet"]
        else:
            return "No results found."
    else:
        return f"Error: {response.status_code}"


In [ ]:
def get_weather(city):
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
    response = requests.get(url).json()
    if "results" not in response:
        return "City not found."

    lat = response["results"][0]["latitude"]
    lon = response["results"][0]["longitude"]

    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    data = requests.get(weather_url).json()
    weather = data["current_weather"]
    return f"{city}: {weather['temperature']}°C, {weather['windspeed']} km/h wind"


In [ ]:
def get_news(topic):
    url = f"https://newsdata.io/api/1/news?apikey={NEWSDATA_API_KEY}&q={topic}&language=en"
    response = requests.get(url).json()

    if "results" in response:
        top_articles = response["results"][:3]
        news_text = "\n\n".join([f"📰 {a['title']}" for a in top_articles])
        return news_text
    else:
        return "No news found."


In [ ]:
def calculator(expression):
    try:
        return str(eval(expression))
    except:
        return "Invalid expression."


In [86]:
def ai_chat(query, conversation_history=None):
    if conversation_history is None:
        conversation_history = []

    greetings = ["hi", "hello", "hey", "good morning", "good evening", "good afternoon"]
    if query.lower().strip() in greetings:
        reply = "Hello! 😊 How can I help you today?"
        conversation_history.append({"user": query, "bot": reply})
        return reply  # ✅ return only the text

    url = "https://google.serper.dev/search"
    headers = {
        "X-API-KEY": SERPER_API_KEY,
        "Content-Type": "application/json"
    }
    payload = {"q": query}

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=10)
        response.raise_for_status()
        results = response.json()
    except Exception as e:
        reply = f"Sorry, I couldn't complete the search because of an error: {e}"
        conversation_history.append({"user": query, "bot": reply})
        return reply

    snippets = []
    if "organic" in results:
        for item in results["organic"][:3]:
            snippet = item.get("snippet", "")
            if snippet:
                snippets.append(snippet)

    info = " ".join(snippets).strip()
    if not info:
        info = "I couldn’t find much on that, but let’s talk about it!"


    reply = (
        f"Here's what I found:\n\n{info}\n\n"
        f"So basically, that’s the gist of it. Do you want to know more about it?"
    )

    conversation_history.append({"user": query, "bot": reply})
    return reply

In [87]:
print("🤖 Chatbot ready! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")
    if user_input.lower().strip() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye! 👋")
        break
    print("Bot:", ai_chat(user_input), "\n")


🤖 Chatbot ready! Type 'exit' to stop.

You: exit
Bot: Goodbye! 👋


In [ ]:
print(ai_chat("Explain artificial intelligence in simple words"))


Here's what I found:

Artificial intelligence is a technology that allows you to generate, classify, and perform tasks like image analysis and speech recognition. AI is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy. Artificial intelligence (AI) refers to computer systems capable of performing complex tasks that historically only humans could do, such as ...

So basically, that’s the gist. Want more on Explain?


In [ ]:
def chatbot(query):
    q = query.lower()
    if "weather" in q:
        city = q.split("weather in")[-1].strip() if "weather in" in q else q.replace("weather", "").strip()
        return get_weather(city or "Dhaka")
    elif "news" in q:
        topic = q.replace("news", "").strip()
        return get_news(topic or "world")
    elif any(op in q for op in ["+", "-", "*", "/", "calculate", "eval"]):
        expr = q.replace("calculate", "").strip()
        return calculator(expr)
    elif "search" in q or "google" in q:
        topic = q.replace("search", "").replace("google", "").strip()
        return google_search(topic)
    else:
        return ai_chat(query)


In [85]:
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="Ask me anything..."),
    outputs=[gr.Textbox(lines=6)],
    title="🌐 Multifunctional Chatbot",
    description="Search, Weather, News, Calculator, and AI Chat — all in one!"
)
iface.launch(share=False, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
